# JON'S HiML Competition RFG_Log Model_picker (v 2.RFG)
# (Decision Tree Version)
## Make Date: 04/13/18

1. This ML model was built using scikit's RandomForestRegressor model.  
1. Automatically find the best combination of input parameters using the combo-picker function    
1. Let's implement log transform on the quantities to improve score.



In [121]:
#Some initialization procedures:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

# load in data files
# FILE_DIR = '../input/hawaiiml-data'
# for f in os.listdir(FILE_DIR):
#     print('{0:<30}{1:0.2f}MB'.format(f, 1e-6*os.path.getsize(f'{FILE_DIR}/{f}')))
df_train = pd.read_csv(f'train_rm.csv', encoding='ISO-8859-1') #write training data to dataframe
df_test = pd.read_csv(f'test_rm.csv', encoding='ISO-8859-1') # Read the test data

#define the error function:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true) - np.log1p(y_pred))**2))

# Build Features

In [122]:
# #convert date strings to datatime datatypes:
# df_train['date'] = pd.to_datetime(df_train['date'])
# df_test['date'] = pd.to_datetime(df_test['date'])

# df_train = df_train.assign(day_of_week = df_train.date.dt.dayofweek)
# df_test = df_test.assign(day_of_week = df_test.date.dt.dayofweek)

In [123]:
# df_train = df_train.assign(is_day0To2 = 0)
# df_train['is_day0To2'] = 0
# df_train.loc[(df_train['day_of_week'] < 3),['is_day0To2']]  = 1

# df_train = df_train.assign(is_day3 = 0)
# df_train['is_day3'] = 0
# df_train.loc[(df_train['day_of_week'] == 3),['is_day3']]  = 1

# df_train = df_train.assign(is_day4 = 0)
# df_train['is_day4'] = 0
# df_train.loc[(df_train['day_of_week'] == 4),['is_day4']]  = 1

# df_train = df_train.assign(is_day6 = 0)
# df_train['is_day6'] = 0
# df_train.loc[(df_train['day_of_week'] == 6),['is_day6']]  = 1

In [139]:
df_train = df_train.loc[(df_train['unit_price'] > 0) & (df_train['unit_price'] < 10000) ]
df_train.loc[df_train['description'].str.match('postage')]

df_train = df_train.assign(unit_price_log1p = np.log1p(df_train.unit_price))
df_test = df_test.assign(unit_price_log1p = np.log1p(df_test.unit_price))


ValueError: labels ['id' 'date' 'time' 'invoice_id' 'stock_id' 'customer_id' 'country'
 'description' 'unit_price' 'quantity' 'UK' 'log_quantity' 'date2' 'time2'
 'hour' 'desc_set' 'desc_bag' 'desc_red' 'desc_heart' 'desc_retrospot'
 'desc_vintage' 'desc_design' 'desc_pink' 'desc_christmas' 'desc_box'
 'morning' 'afternoon' 'evening' 'mean_price' 'max_price' 'min_price'
 'median_price' 'median_diff' 'mean_diff' 'min_diff' 'max_diff'
 'unit_price_log1p' 'customer_7' 'is_uk' 'is_free' 'is_postage'
 'is_dotcompostage'] not contained in axis

In [126]:
df_train = df_train.assign(customer_7 = 0)
df_train.loc[(df_train['customer_id'] == 7),['customer_7']]  = 1

df_train = df_train.assign(is_uk = 0)
df_train.loc[(df_train['country'] == "united kingdom"),['is_uk']]  = 1

df_train = df_train.assign(is_free = 0)
df_train.loc[(df_train['unit_price'] == 0),['is_free']]  = 1

df_train = df_train.assign(is_postage = 0)
df_train.loc[(df_train['description'].str.match('postage')),['is_postage']]  = 1

df_train = df_train.assign(is_dotcompostage = 0)
df_train.loc[(df_train['description'].str.match('dotcom postage')),['is_dotcompostage']]  = 1

In [137]:
df_train.head()

,id,date,time,invoice_id,stock_id,customer_id,country,description,unit_price,quantity,...,median_diff,mean_diff,min_diff,max_diff,unit_price_log1p,customer_7,is_uk,is_free,is_postage,is_dotcompostage
305,282662,1/24/2011,10:12:00,5268,95,2191,germany,postage,18.0,2,...,0.0,-15.216955,18.0,8124.75,2.944439,0,0,0,1,0
758,520198,5/9/2011,09:47:00,5807,95,111,france,postage,18.0,2,...,0.0,-15.216955,18.0,8124.75,2.944439,0,0,0,1,0
816,41011,9/16/2011,14:20:00,12761,95,1874,germany,postage,18.0,1,...,0.0,-15.216955,18.0,8124.75,2.944439,0,0,0,1,0
844,233086,1/31/2011,09:05:00,6158,95,2436,germany,postage,18.0,3,...,0.0,-15.216955,18.0,8124.75,2.944439,0,0,0,1,0
1629,343450,3/17/2011,13:23:00,9547,95,182,germany,postage,18.0,7,...,0.0,-15.216955,18.0,8124.75,2.944439,0,0,0,1,0


In [83]:
postage[postage['description'] == "dotcom postage"]

,id,date,time,invoice_id,stock_id,customer_id,country,description,unit_price,quantity,...,median_price,median_diff,mean_diff,min_diff,max_diff,unit_price_log1p,customer_7,is_uk,is_free,is_postage
305,282662,1/24/2011,10:12:00,5268,95,2191,germany,postage,18.00,2,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
758,520198,5/9/2011,09:47:00,5807,95,111,france,postage,18.00,2,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
816,41011,9/16/2011,14:20:00,12761,95,1874,germany,postage,18.00,1,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
844,233086,1/31/2011,09:05:00,6158,95,2436,germany,postage,18.00,3,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
1629,343450,3/17/2011,13:23:00,9547,95,182,germany,postage,18.00,7,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
1670,388317,11/16/2011,16:06:00,11464,95,251,germany,postage,18.00,2,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
1743,57765,6/20/2011,12:14:00,239,95,169,denmark,postage,18.00,9,...,18.0,0.00,-15.216955,18.00,8124.75,2.944439,0,0,0,1
1797,178598,2/4/2011,10:38:00,8820,95,3002,switzerland,postage,40.00,4,...,18.0,22.00,6.783045,40.00,8102.75,3.713572,0,0,0,1
2294,245319,8/5/2011,13:52:00,4018,95,200,italy,postage,28.00,4,...,18.0,10.00,-5.216955,28.00,8114.75,3.367296,0,0,0,1
2513,426823,6/13/2011,10:54:00,12025,95,366,united kingdom,postage,2.02,1,...,18.0,-15.98,-31.196955,2.02,8140.73,1.105257,0,1,0,1


In [47]:
df_test[df_test['unit_price'] == 0]

,id,date,time,invoice_id,stock_id,customer_id,country,description,unit_price,UK,...,evening,mean_price,max_price,min_price,median_price,median_diff,mean_diff,min_diff,max_diff,unit_price_log1p
249,186209,12/22/2010,14:41:00,813,659,7,united kingdom,glass jar daisy fresh cotton wool,0.0,1,...,0,4.431477,5.91,0.0,5.790,-5.790,-4.431477,0.0,5.91,0.0
1205,475385,11/22/2011,12:09:00,3131,1464,7,united kingdom,check,0.0,1,...,0,2.371667,4.13,0.0,2.080,-2.080,-2.371667,0.0,4.13,0.0
1412,384135,3/8/2011,17:43:00,3519,1103,7,united kingdom,NaN,0.0,1,...,0,3.376000,4.25,0.0,4.240,-4.240,-3.376000,0.0,4.25,0.0
1462,307544,1/11/2011,09:14:00,3607,1875,7,united kingdom,NaN,0.0,1,...,0,1.983333,5.95,0.0,0.000,0.000,-1.983333,0.0,5.95,0.0
1616,228513,6/21/2011,13:12:00,3889,1958,7,united kingdom,NaN,0.0,1,...,0,3.400000,4.25,0.0,4.250,-4.250,-3.400000,0.0,4.25,0.0
2369,64777,9/5/2011,12:14:00,5155,95,7,united kingdom,NaN,0.0,1,...,0,25.557530,700.00,0.0,18.000,-18.000,-25.557530,0.0,700.00,0.0
2407,275826,4/6/2011,15:26:00,5205,2258,7,united kingdom,NaN,0.0,1,...,0,0.337778,0.38,0.0,0.380,-0.380,-0.337778,0.0,0.38,0.0
2571,125359,12/22/2010,14:41:00,813,2195,7,united kingdom,childs garden spade blue,0.0,1,...,0,2.336765,4.13,0.0,2.100,-2.100,-2.336765,0.0,4.13,0.0
3511,276427,11/15/2011,14:55:00,6673,1375,7,united kingdom,NaN,0.0,1,...,0,0.614318,4.13,0.0,0.390,-0.390,-0.614318,0.0,4.13,0.0
4240,193046,11/24/2011,15:56:00,7485,1588,7,united kingdom,NaN,0.0,1,...,0,6.429559,11.02,0.0,4.950,-4.950,-6.429559,0.0,11.02,0.0


In [48]:
# df_train = df_train.assign(uplog1p_quant1 = 0)
# df_train.loc[(df_train['unit_price_log1p'] < 6.47774129795256),['uplog1p_quant1']]  = 1
# df_test = df_test.assign(uplog1p_quant1 = 0)
# df_test.loc[(df_test['unit_price_log1p'] < 6.47774129795256),['uplog1p_quant1']]  = 1

In [49]:
# df_train['time'] = pd.to_datetime(df_train['time'])
# df_test['time'] = pd.to_datetime(df_test['time'])
# df_train = df_train.assign(hr = df_train.time.dt.hour)
# df_test = df_test.assign(hr = df_test.time.dt.hour)

In [50]:
# df_train = df_train.assign(invoice_id_log1p = np.log1p(df_train.invoice_id))
# df_test = df_test.assign(invoice_id_log1p = np.log1p(df_test.invoice_id))

# Define Training Target Data:
1. We want to predict the quantity data field.    
1. By convention, we define this target as 'y'.  
1. From model 2.02, we know that doing a log transform on the tartget reduces skewness and improves score.  

In [127]:
y = df_train.quantity
logy = np.log1p(df_train.quantity) #take log of quantities

# Define ML Predictors
build a function to go through each combination of predictors

### Here is the list of columns we can choose predictors from. 

In [128]:
print('Column Names & Data Types: \n', df_train.dtypes)

Column Names & Data Types: 
 id                    int64
date                 object
time                 object
invoice_id            int64
stock_id              int64
customer_id           int64
country              object
description          object
unit_price          float64
quantity              int64
UK                    int64
log_quantity        float64
date2                object
time2                object
hour                  int64
desc_set              int64
desc_bag              int64
desc_red              int64
desc_heart            int64
desc_retrospot        int64
desc_vintage          int64
desc_design           int64
desc_pink             int64
desc_christmas        int64
desc_box              int64
morning               int64
afternoon             int64
evening               int64
mean_price          float64
max_price           float64
min_price           float64
median_price        float64
median_diff         float64
mean_diff           float64
min_diff           

## Define feature combination picker

In [129]:
# ls_AllPredictors = ['invoice_id', 'stock_id',
#     'desc_set','desc_bag','desc_red','desc_heart','desc_retrospot','desc_vintage',
#     'desc_design','desc_pink','desc_christmas','desc_box',
#     'morning','afternoon','evening']
                   
ls_AllPredictors = ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_bag', 'desc_box', 'evening', 'stock_id','invoice_id']



# https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
#build combos:
ls_PredictorCombos = [list(combo[1]) for combo in enumerate(powerset(ls_AllPredictors), 1)]
#Define combo getting function:
def GetX(comboID, adf_train):
    #get X values in adf_train for the given list of predictors
    return adf_train[ls_PredictorCombos[comboID]]

# Implement Random Forest Regressor Model on Each Predictor Combo
## Define Some Helper Functions

In [130]:
def Run_RFG(SeedVal, X, logy):
    #fit, predict, and then evaluate a model using passed in values (e.g. training set)
    #return back RMSLE value and trained model

    myModel = DecisionTreeRegressor()

    train_X, val_X, train_y, val_y = train_test_split(X, logy,random_state = SeedVal) #split training data into a test and train part
    myModel.fit(train_X, train_y)
    predicted_vals = np.expm1(myModel.predict(val_X)) #transform predicted values from log to "normal" Y value
    return rmsle(np.expm1(val_y), predicted_vals), myModel #include transform of val_Y values from log to "normal" Y value

In [131]:
def FindBestCombo_Model(adf_train, logy):
    #iteratively search for best parameter combinations to match X values in adf_train with logy
    #return best combo, error results of each model, training error, and model
    MinErr = 100000000
    df_Track = pd.DataFrame()
    for comboID in range(1,len(ls_PredictorCombos)):
        print('Eval combo:', ls_PredictorCombos[comboID])
        SeedVal = 0
        X = GetX(comboID, adf_train)
        TrainErr,myModel = Run_RFG(SeedVal,X,logy)
        df2 = pd.DataFrame([[comboID, TrainErr, ','.join(ls_PredictorCombos[comboID])]],columns=['ComboID','err','Preds'])
        if df_Track.shape[0] >0:
            df_Track = pd.concat([df2, df_Track])
        else:
            df_Track = df2.copy(deep = True)
        if TrainErr < MinErr:
            MinErr = TrainErr
    #         BestSeedVal = SeedVal
            bestComboID = comboID
            print ('Best Combo: ', comboID, ' Params: ', ls_PredictorCombos[comboID], ' Err: ', TrainErr)
    #train with best combo:
    X = GetX(bestComboID, adf_train)
    TrainErr,myModel = Run_RFG(0,X,logy)
    print ('fin')
    return ls_PredictorCombos[bestComboID], df_Track, TrainErr, myModel

### Build Customer 7 and Everyone Else Models:
1. Find Best Predictors
1. Define Separate Models

In [132]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff'] ],logy)
TrainErr

0.46564210070131024

In [133]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff',"desc_bag","evening"] ],logy)
TrainErr

0.46587818440707152

In [134]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff', 'is_uk','customer_7'] ],logy)
TrainErr

0.46363307522647101

In [135]:
TrainErr,myModel = Run_RFG(0,df_train[['is_postage','customer_id', 'unit_price_log1p', 'median_diff', 'is_uk'] ],logy)
TrainErr

0.46366322418821643

In [136]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff', 'is_uk','is_postage'] ],logy)
TrainErr

0.46366322418821643

In [16]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff', 'customer_7','desc_bag'] ],logy)
TrainErr

0.58497953921804358

In [19]:
TrainErr,myModel = Run_RFG(0,df_train[['customer_id', 'unit_price_log1p', 'median_diff', 'customer_7','desc_bag','evening'] ],logy)
TrainErr

0.58677837432293722

In [72]:
ls_mypredictors, df_Track, TrainErr, myModel = FindBestCombo_Model(df_train, logy)

Eval combo: ['customer_id']
Best Combo:  1  Params:  ['customer_id']  Err:  0.702305575779
Eval combo: ['unit_price_log1p']
Eval combo: ['median_diff']
Eval combo: ['desc_bag']
Eval combo: ['desc_box']
Eval combo: ['evening']
Eval combo: ['stock_id']
Eval combo: ['invoice_id']
Best Combo:  8  Params:  ['invoice_id']  Err:  0.6834482337
Eval combo: ['customer_id', 'unit_price_log1p']
Best Combo:  9  Params:  ['customer_id', 'unit_price_log1p']  Err:  0.601746174119
Eval combo: ['customer_id', 'median_diff']
Eval combo: ['customer_id', 'desc_bag']
Eval combo: ['customer_id', 'desc_box']
Eval combo: ['customer_id', 'evening']
Eval combo: ['customer_id', 'stock_id']
Eval combo: ['customer_id', 'invoice_id']
Eval combo: ['unit_price_log1p', 'median_diff']
Eval combo: ['unit_price_log1p', 'desc_bag']
Eval combo: ['unit_price_log1p', 'desc_box']
Eval combo: ['unit_price_log1p', 'evening']
Eval combo: ['unit_price_log1p', 'stock_id']
Eval combo: ['unit_price_log1p', 'invoice_id']
Eval combo: [

Eval combo: ['unit_price_log1p', 'desc_bag', 'stock_id', 'invoice_id']
Eval combo: ['unit_price_log1p', 'desc_box', 'evening', 'stock_id']
Eval combo: ['unit_price_log1p', 'desc_box', 'evening', 'invoice_id']
Eval combo: ['unit_price_log1p', 'desc_box', 'stock_id', 'invoice_id']
Eval combo: ['unit_price_log1p', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['median_diff', 'desc_bag', 'desc_box', 'evening']
Eval combo: ['median_diff', 'desc_bag', 'desc_box', 'stock_id']
Eval combo: ['median_diff', 'desc_bag', 'desc_box', 'invoice_id']
Eval combo: ['median_diff', 'desc_bag', 'evening', 'stock_id']
Eval combo: ['median_diff', 'desc_bag', 'evening', 'invoice_id']
Eval combo: ['median_diff', 'desc_bag', 'stock_id', 'invoice_id']
Eval combo: ['median_diff', 'desc_box', 'evening', 'stock_id']
Eval combo: ['median_diff', 'desc_box', 'evening', 'invoice_id']
Eval combo: ['median_diff', 'desc_box', 'stock_id', 'invoice_id']
Eval combo: ['median_diff', 'evening', 'stock_id', 'invoice_id']
Eval

Eval combo: ['unit_price_log1p', 'median_diff', 'desc_bag', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['unit_price_log1p', 'median_diff', 'desc_box', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['unit_price_log1p', 'desc_bag', 'desc_box', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['median_diff', 'desc_bag', 'desc_box', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_bag', 'desc_box', 'evening', 'stock_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_bag', 'desc_box', 'evening', 'invoice_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_bag', 'desc_box', 'stock_id', 'invoice_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_bag', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'median_diff', 'desc_box', 'evening', 'stock_id', 'invoice_id']
Eval combo: ['customer_id', 'unit_price_log1p', 'desc_bag', 'des

## Observations

In [69]:
display(df_Track.sort_values(by=['err']))

NameError: name 'display' is not defined

In [73]:
df_Track.sort_values(by=['err'])

,ComboID,err,Preds
0,37,0.584898,"customer_id,unit_price_log1p,median_diff"
0,93,0.584899,"customer_id,unit_price_log1p,median_diff,desc_bag"
0,95,0.586089,"customer_id,unit_price_log1p,median_diff,evening"
0,164,0.586582,"customer_id,unit_price_log1p,median_diff,desc_..."
0,94,0.586701,"customer_id,unit_price_log1p,median_diff,desc_box"
0,163,0.587524,"customer_id,unit_price_log1p,median_diff,desc_..."
0,167,0.587676,"customer_id,unit_price_log1p,median_diff,desc_..."
0,219,0.588838,"customer_id,unit_price_log1p,median_diff,desc_..."
0,9,0.601746,"customer_id,unit_price_log1p"
0,40,0.601920,"customer_id,unit_price_log1p,evening"


In [162]:
print (myModel_7.feature_importances_)
print (myModel_no7.feature_importances_)

[ 0.64557884  0.34338557  0.01103559]
[ 0.62476342  0.37523658  0.        ]
